# Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Read csv

In [33]:
data = pd.read_csv("../data/movie_metadata.csv")

# Dropping columns

Some columns do not effect the users choice for movies, we drop those columns to reduce the number of attricutes (features) to deal with.

In [34]:
drop_col = ['color', 'facenumber_in_poster', 'plot_keywords', 'movie_imdb_link', 'aspect_ratio', 
            'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_facebook_likes', 'actor_1_facebook_likes', 
            'gross', 'num_critic_for_reviews', 'num_voted_users', 'num_user_for_reviews', 'budget']
imdb = data.drop(drop_col, axis=1)

In [35]:
imdb.columns

Index(['director_name', 'duration', 'actor_2_name', 'genres', 'actor_1_name',
       'movie_title', 'cast_total_facebook_likes', 'actor_3_name', 'language',
       'country', 'content_rating', 'title_year', 'imdb_score',
       'movie_facebook_likes'],
      dtype='object')

# Convert string to non-string

For the algorithms that we use, string features are not supported. We convert these string attributes to better suit the algorithms and thus remove inconsistencies in the data.

In [36]:
imdb.select_dtypes(exclude=['float', 'int']).columns.values

array(['director_name', 'actor_2_name', 'genres', 'actor_1_name',
       'movie_title', 'actor_3_name', 'language', 'country',
       'content_rating'], dtype=object)

In [37]:
imdb['movie_title'] = imdb['movie_title'].str.strip() # fix movie names (remove \0 and redundant spaces)

### Name values

Name values like actor names and director names are converted to unique ID values. These ID's are then mapped to the names in the original table.

In [38]:
def convert_to_unique_id(col_name, new_col_name, fill_na_as='unknown'):
    global imdb
    # fill all Na values 
    imdb[col_name].fillna(fill_na_as, inplace=True)
    
    # get unique values and create an array with values as index
    elements = imdb[col_name].str.strip().unique()
    _ids = pd.Series(np.arange(len(elements)), index=elements)
    
    # map 
    imdb[new_col_name] = imdb[col_name].apply(lambda x: _ids[x])
    imdb.drop(col_name, axis=1, inplace=True)

In [39]:
# convert names to ids
str_cols = ['director_name', 'actor_2_name', 'actor_1_name', 'actor_3_name']
for col in str_cols:
    new_name = '_'.join(col.split('_')[:-1]) + '_id'
    convert_to_unique_id(col, new_name)

Data after converting to unique ID's

In [40]:
imdb.head(3)

,duration,genres,movie_title,cast_total_facebook_likes,language,country,content_rating,title_year,imdb_score,movie_facebook_likes,director_id,actor_2_id,actor_1_id,actor_3_id
0,178.0,Action|Adventure|Fantasy|Sci-Fi,Avatar,4834,English,USA,PG-13,2009.0,7.9,33000,0,0,0,0
1,169.0,Action|Adventure|Fantasy,Pirates of the Caribbean: At World's End,48350,English,USA,PG-13,2007.0,7.1,0,1,1,1,1
2,148.0,Action|Adventure|Thriller,Spectre,11700,English,UK,PG-13,2015.0,6.8,85000,2,2,2,2


To make sure, for director ID == 0 (James Cameron), we can see the movies he has directed.

In [41]:
imdb[imdb.director_id == 0]

,duration,genres,movie_title,cast_total_facebook_likes,language,country,content_rating,title_year,imdb_score,movie_facebook_likes,director_id,actor_2_id,actor_1_id,actor_3_id
0,178.0,Action|Adventure|Fantasy|Sci-Fi,Avatar,4834,English,USA,PG-13,2009.0,7.9,33000,0,0,0,0
26,194.0,Drama|Romance,Titanic,45223,English,USA,PG-13,1997.0,7.7,26000,0,23,20,23
288,153.0,Action|Sci-Fi,Terminator 2: Judgment Day,2829,English,USA,R,1991.0,8.5,13000,0,224,155,251
291,141.0,Action|Comedy|Thriller,True Lies,4528,English,USA,R,1994.0,7.2,0,0,226,157,253
606,171.0,Adventure|Drama|Sci-Fi|Thriller,The Abyss,4074,English,USA,PG-13,1989.0,7.6,0,0,446,280,511
2486,154.0,Action|Adventure|Sci-Fi,Aliens,4228,English,USA,R,1986.0,8.4,18000,0,1511,280,1804
3575,107.0,Action|Sci-Fi,The Terminator,3582,English,UK,R,1984.0,8.1,13000,0,588,280,1834


### Country name and language
#### Remove null vales
All movies that have null as thier 'country' or 'language', are actually US movie. We fill them with accordingly

In [42]:
imdb[(imdb.country.isnull() | imdb.language.isnull())][['movie_title', 'country', 'language']]

,movie_title,country,language
4,Star Wars: Episode VII - The Force Awakens,NaN,NaN
279,"10,000 B.C.",NaN,NaN
2370,"Gone, Baby, Gone",NaN,English
2459,Unforgettable,USA,NaN
3086,September Dawn,USA,NaN
3397,Preacher,NaN,English
3539,Alpha and Omega 4: The Legend of the Saw Tooth...,USA,NaN
3869,Silent Movie,USA,NaN
4021,Dawn Patrol,NaN,English
4110,Love's Abiding Joy,USA,NaN


In [43]:
imdb.country.fillna('USA', inplace=True)
imdb.language.fillna('English', inplace=True)

#### Replace with unique IDs

In [44]:
for col in ['language', 'country']:
    convert_to_unique_id(col_name=col, new_col_name=col+'_id')

In [45]:
imdb.head(3)

,duration,genres,movie_title,cast_total_facebook_likes,content_rating,title_year,imdb_score,movie_facebook_likes,director_id,actor_2_id,actor_1_id,actor_3_id,language_id,country_id
0,178.0,Action|Adventure|Fantasy|Sci-Fi,Avatar,4834,PG-13,2009.0,7.9,33000,0,0,0,0,0,0
1,169.0,Action|Adventure|Fantasy,Pirates of the Caribbean: At World's End,48350,PG-13,2007.0,7.1,0,1,1,1,1,0,0
2,148.0,Action|Adventure|Thriller,Spectre,11700,PG-13,2015.0,6.8,85000,2,2,2,2,0,1


### Content rating

The content ratings are spread over a wide spectrum, we map similar values to reduce dimentinality in data

Movies - PG-13, PG, G, R, NC-17<br>
TV - TV-14, TV-PG, TV-MA, TV-G, TV-Y, TV-Y7<br>
others: Approved, Passed<br>
nulls - nan, Not Rated, Unrated, X<br>

map as:<br>

|new-rating|includes|
|----------|--------|
|G|G, TV-Y, TV-Y7, TV-G
|PG|PG, TV-PG, Approved, Passed, GP
|PG-13|PG-13, TV-14
|R|R, X
|NC-17|NC-17, TV-MA, M
|unrated| nan, Not Rated, Unrated
    

In [46]:
imdb.content_rating.unique() # all unique content rating values in the data

array(['PG-13', nan, 'PG', 'G', 'R', 'TV-14', 'TV-PG', 'TV-MA', 'TV-G',
       'Not Rated', 'Unrated', 'Approved', 'TV-Y', 'NC-17', 'X', 'TV-Y7',
       'GP', 'Passed', 'M'], dtype=object)

In [47]:
imdb['content_rating'] = imdb.content_rating.replace({
    'TV-Y' : 'G',
    'TV-Y7' : 'G',
    'TV-G' : 'G',
    'TV-PG' : 'PG',
    'Approved' : 'PG',
    'Passed' : 'PG',
    'GP' : 'PG',
    'TV-14' : 'PG-13',
    'X' : 'R',
    'TV-MA' : 'NC-17',
    'M' : 'NC-17',
    'Not Rated' : 'Unrated'
})

In [48]:
imdb.content_rating.fillna('Unrated', inplace=True)

In [50]:
imdb.content_rating.unique() # new content rating values after mapping

array(['PG-13', 'Unrated', 'PG', 'G', 'R', 'NC-17'], dtype=object)

#### Convert to unique IDs

In [52]:
convert_to_unique_id(col_name='content_rating', new_col_name=col+'_id')
imdb.head(3)

,duration,genres,movie_title,cast_total_facebook_likes,title_year,imdb_score,movie_facebook_likes,director_id,actor_2_id,actor_1_id,actor_3_id,language_id,country_id
0,178.0,Action|Adventure|Fantasy|Sci-Fi,Avatar,4834,2009.0,7.9,33000,0,0,0,0,0,0
1,169.0,Action|Adventure|Fantasy,Pirates of the Caribbean: At World's End,48350,2007.0,7.1,0,1,1,1,1,0,0
2,148.0,Action|Adventure|Thriller,Spectre,11700,2015.0,6.8,85000,2,2,2,2,0,0


### Genre

The genre for each movie are hardcoded in the 'genre' column with a pipe ( | ) separater. We pares all the genre values and assign each to a new column. Each genre column is a binary variable showing stating if the movie is under that preticular genre.

In [53]:
imdb.iloc[2] # genre for movie Spectre can be seen as Action|Adventure|Thriller

duration                                           148
genres                       Action|Adventure|Thriller
movie_title                                    Spectre
cast_total_facebook_likes                        11700
title_year                                        2015
imdb_score                                         6.8
movie_facebook_likes                             85000
director_id                                          2
actor_2_id                                           2
actor_1_id                                           2
actor_3_id                                           2
language_id                                          0
country_id                                           0
Name: 2, dtype: object

In [54]:
# helper function to parse the genres
def genre_parser(row):
    genres = row.genres.split('|')
    for genre in genres:
        row.set_value(genre.strip(), 1)
    return row

In [55]:
genres = set()
imdb['genres'].apply(lambda x: genres.update(x.split('|')))

print('Total number of genres: {}'.format(len(genres)))
print('Genres are: ', [g for g in genres], sep=' ')

Total number of genres: 26
Genres are:  ['Crime', 'Sport', 'Documentary', 'Family', 'Horror', 'Mystery', 'Action', 'Adventure', 'Sci-Fi', 'Romance', 'Short', 'News', 'Animation', 'Musical', 'Game-Show', 'Biography', 'Western', 'Reality-TV', 'Comedy', 'War', 'Thriller', 'Drama', 'Film-Noir', 'History', 'Music', 'Fantasy']


We add new columns to the data frame with the column names are the genres. Then use binary encoding to specify the genres of the movie

In [56]:
for genre in genres:
    imdb[genre.strip()] = 0

In [57]:
imdb.columns # all columns after parsing and making new columns

Index(['duration', 'genres', 'movie_title', 'cast_total_facebook_likes',
       'title_year', 'imdb_score', 'movie_facebook_likes', 'director_id',
       'actor_2_id', 'actor_1_id', 'actor_3_id', 'language_id', 'country_id',
       'Crime', 'Sport', 'Documentary', 'Family', 'Horror', 'Mystery',
       'Action', 'Adventure', 'Sci-Fi', 'Romance', 'Short', 'News',
       'Animation', 'Musical', 'Game-Show', 'Biography', 'Western',
       'Reality-TV', 'Comedy', 'War', 'Thriller', 'Drama', 'Film-Noir',
       'History', 'Music', 'Fantasy'],
      dtype='object')

In [58]:
imdb = imdb.apply(genre_parser, axis=1) # apply parser function for each row

In [62]:
print("Genres for {}: {}".format(imdb.iloc[0].movie_title, imdb.iloc[0].genres))
print("\n Genres columns for the movie: ")
imdb.iloc[0][19:]

Genres for Avatar: Action|Adventure|Fantasy|Sci-Fi

 Genres columns for the movie: 


Action        1
Adventure     1
Sci-Fi        1
Romance       0
Short         0
News          0
Animation     0
Musical       0
Game-Show     0
Biography     0
Western       0
Reality-TV    0
Comedy        0
War           0
Thriller      0
Drama         0
Film-Noir     0
History       0
Music         0
Fantasy       1
Name: 0, dtype: object

In [63]:
imdb = imdb.drop('genres', axis=1) # drop genre column

In [64]:
imdb.columns

Index(['duration', 'movie_title', 'cast_total_facebook_likes', 'title_year',
       'imdb_score', 'movie_facebook_likes', 'director_id', 'actor_2_id',
       'actor_1_id', 'actor_3_id', 'language_id', 'country_id', 'Crime',
       'Sport', 'Documentary', 'Family', 'Horror', 'Mystery', 'Action',
       'Adventure', 'Sci-Fi', 'Romance', 'Short', 'News', 'Animation',
       'Musical', 'Game-Show', 'Biography', 'Western', 'Reality-TV', 'Comedy',
       'War', 'Thriller', 'Drama', 'Film-Noir', 'History', 'Music', 'Fantasy'],
      dtype='object')

## Fix numeric values

In [43]:
num_cols = imdb.select_dtypes(include=['float', 'int']).columns[:-30]
num_cols

Index(['num_critic_for_reviews', 'duration', 'gross', 'num_voted_users',
       'cast_total_facebook_likes', 'num_user_for_reviews', 'budget',
       'title_year', 'imdb_score', 'movie_facebook_likes'],
      dtype='object')

We will drop <br>
    1. 'gross',
    2. 'num_critic_for_reviews'
    3. 'num_voted_users'
    4. 'num_user_for_reviews'
    5. 'budget'
as these do not affect user liking/disliking a movie<br>

In [44]:
imdb.drop(
    ['gross', 'num_critic_for_reviews', 'num_voted_users', 'num_user_for_reviews', 'budget'],
    axis=1,
    inplace=True
)

In [47]:
imdb.columns

Index(['duration', 'movie_title', 'cast_total_facebook_likes', 'language',
       'country', 'content_rating', 'title_year', 'imdb_score',
       'movie_facebook_likes', 'director_id', 'actor_2_id', 'actor_1_id',
       'actor_3_id', 'Short', 'News', 'Documentary', 'History', 'Biography',
       'Game-Show', 'Musical', 'Animation', 'Mystery', 'Comedy', 'Music',
       'Romance', 'Adventure', 'Fantasy', 'Drama', 'Thriller', 'Sport',
       'Horror', 'Action', 'Reality-TV', 'Sci-Fi', 'Crime', 'War', 'Western',
       'Family', 'Film-Noir'],
      dtype='object')

In [48]:
imdb.isnull().sum()[:-30]

duration                      15
movie_title                    0
cast_total_facebook_likes      0
language                       0
country                        0
content_rating                 0
title_year                   108
imdb_score                     0
movie_facebook_likes           0
dtype: int64

### fix duration and title_year

In [49]:
imdb[imdb.duration.isnull()]['movie_title']

4          Star Wars: Episode VII - The Force Awakens
199     Harry Potter and the Deathly Hallows: Part II
206      Harry Potter and the Deathly Hallows: Part I
1510                              Black Water Transit
3604                                      War & Peace
3815                             Should've Been Romeo
3834                                            Barfi
4299                           Hum To Mohabbat Karega
4392                                         N-Secure
4397                              Dil Jo Bhi Kahey...
4517                                       Wolf Creek
4609                                Karachi se Lahore
4690                                          Destiny
4948                                 Romantic Schemer
4989                                    The Naked Ape
Name: movie_title, dtype: object

Fill with mean!

In [50]:
imdb.duration.fillna(imdb.duration.mean(), inplace=True)

IDK what to do with title year, fill with 0 for now!

In [51]:
imdb.title_year.fillna(0, inplace=True)

## Final checks!

In [52]:
imdb.isnull().sum().sum() 

0

In [54]:
imdb.select_dtypes(exclude=['float', 'int']).columns

Index(['movie_title', 'language', 'country', 'content_rating'], dtype='object')

In [55]:
# rename language, country, and content_rating with unique ids
for col in ['language', 'country', 'content_rating']:
    convert_to_unique_id(col_name=col, new_col_name=col+'_id')

In [56]:
imdb.select_dtypes(exclude=['float', 'int']).columns

Index(['movie_title'], dtype='object')

In [57]:
movies = imdb.movie_title
imdb.drop('movie_title', axis=1, inplace=True)

In [58]:
imdb.set_index(movies)

,duration,cast_total_facebook_likes,title_year,imdb_score,movie_facebook_likes,director_id,actor_2_id,actor_1_id,actor_3_id,Short,...,Reality-TV,Sci-Fi,Crime,War,Western,Family,Film-Noir,language_id,country_id,content_rating_id
movie_title,,,,,,,,,,,,,,,,,,,,,
Avatar,178.000000,4834,2009.0,7.9,33000,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
Pirates of the Caribbean: At World's End,169.000000,48350,2007.0,7.1,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
Spectre,148.000000,11700,2015.0,6.8,85000,2,2,2,2,0,...,0,0,0,0,0,0,0,0,1,0
The Dark Knight Rises,164.000000,106759,2012.0,8.5,164000,3,3,3,3,0,...,0,0,0,0,0,0,0,0,0,0
Star Wars: Episode VII - The Force Awakens,107.201074,143,0.0,7.1,0,4,4,4,4,0,...,0,0,0,0,0,0,0,0,0,1
John Carter,132.000000,1873,2012.0,6.6,24000,5,5,5,5,0,...,0,1,0,0,0,0,0,0,0,0
Spider-Man 3,156.000000,46055,2007.0,6.2,0,6,6,6,6,0,...,0,0,0,0,0,0,0,0,0,0
Tangled,100.000000,2036,2010.0,7.8,29000,7,7,7,7,0,...,0,0,0,0,0,1,0,0,0,2
Avengers: Age of Ultron,141.000000,92000,2015.0,7.5,118000,8,8,8,8,0,...,0,1,0,0,0,0,0,0,0,0


Convert unnecessary float dtypes to int

In [59]:
imdb.select_dtypes(include=['float']).columns

Index(['duration', 'title_year', 'imdb_score'], dtype='object')

In [60]:
imdb['duration'] = imdb['duration'].astype('int16')
imdb['title_year'] = imdb['title_year'].astype('int16')

In [61]:
imdb.duration.dtype, imdb.title_year.dtype

(dtype('int16'), dtype('int16'))

In [62]:
imdb.head()

,duration,cast_total_facebook_likes,title_year,imdb_score,movie_facebook_likes,director_id,actor_2_id,actor_1_id,actor_3_id,Short,...,Reality-TV,Sci-Fi,Crime,War,Western,Family,Film-Noir,language_id,country_id,content_rating_id
0,178,4834,2009,7.9,33000,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,169,48350,2007,7.1,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,148,11700,2015,6.8,85000,2,2,2,2,0,...,0,0,0,0,0,0,0,0,1,0
3,164,106759,2012,8.5,164000,3,3,3,3,0,...,0,0,0,0,0,0,0,0,0,0
4,107,143,0,7.1,0,4,4,4,4,0,...,0,0,0,0,0,0,0,0,0,1


## Save dataset for later use

In [102]:
imdb['movie_title'] = movies

In [103]:
imdb.to_csv('../data/imdb_dataset.csv', index=False)

## Helper to load data

In [104]:
def load_imdb(path):
    data = pd.read_csv(path)
    movies = data['movie_title']
    data.drop('movie_title', axis=1, inplace=True)
    data.set_index(movies, inplace=True)
    return movies, data

In [105]:
movies, imdb = load_imdb('../data/imdb_dataset.csv')

In [106]:
imdb.head()

,duration,cast_total_facebook_likes,title_year,imdb_score,movie_facebook_likes,director_id,actor_2_id,actor_1_id,actor_3_id,Action,...,Horror,Family,Mystery,Biography,Crime,History,Romance,language_id,country_id,content_rating_id
movie_title,,,,,,,,,,,,,,,,,,,,,
Avatar,178,4834,2009,7.9,33000,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Pirates of the Caribbean: At World's End,169,48350,2007,7.1,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Spectre,148,11700,2015,6.8,85000,2,2,2,2,1,...,0,0,0,0,0,0,0,0,1,0
The Dark Knight Rises,164,106759,2012,8.5,164000,3,3,3,3,1,...,0,0,0,0,0,0,0,0,0,0
Star Wars: Episode VII - The Force Awakens,107,143,0,7.1,0,4,4,4,4,0,...,0,0,0,0,0,0,0,0,0,1
